In [1]:
import torch
import pyvene as pv
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def zeroout_intervention(b, s):
    b[:, 3] = 0.
    return b

In [ ]:
pv_model = pv.IntervenableModel({
    "component": "model.layers[15].mlp.output",
    "intervention": zeroout_intervention
}, model=model)

In [ ]:
orig_outputs, intervened_outputs = pv_model(
    tokenizer("the capital of spain is", return_tensors='pt').to('cuda'),
    output_original_output=True
)
print(intervened_outputs.logits - orig_outputs.logits)

In [ ]:
pv_model = pv.IntervenableModel.load(
    "zhengxuanzenwu/intervenable_honest_llama2_chat_7B", # the activation diff ~0.14MB
    model,
)